Skeleton code for resolution theorem prover for propositional logic

Yoonsuck Choe  
20 October 2021

clause representation:  
```
[<positive-proposition-list>, <negative-proposition-list>]
```

proposition-list representation:  
```
[ 'p', 'q', 'r' ]
```

Note: `p`, `q`, `r`, `s` are propositions : CASE SENSITIVE!

Example clause:  
`[['p', 'q'] ['r']]` : This is clause P &#x2228; Q &#x2228; &#x00ac;R

In [ ]:
DEBUG = True

In [ ]:
def rm_item(lst, item):
    '''
    function: rm_pred()

    remove item from list
    - return [] if empty, not None
    '''
    lst.remove(item)

    if lst == None:
        return []
    else:
        return lst

In [ ]:
def mk_unique(clause):
    '''
    remove redundant propositions in the clause
    '''
    pos = clause[0]
    neg = clause[1]

    pos = list(set(pos))
    neg = list(set(neg))

    return [pos, neg]

In [ ]:
def resolve(clause1, clause2):
    '''
    function: resolve()

    [['p', 'q'] ['r']] : clause P \/ Q \/ ~R

    resolve clauses c1 and c2
    - if resolvable returns resolvent
    - if not resolvable returns False
    - returns empty clause [[], []] when Falsified

    '''

    # remove any redundant propositions in the clauses
    c1 = mk_unique(clause1)
    c2 = mk_unique(clause2)

    count = 0

    # print clauses
    if DEBUG:
        print("resolving: "+str(c1)+" and "+str(c2))

    # c1 pos vs. c2 neg
    for p1 in c1[0]:
        for p2 in c2[1]:
            if p1 == p2:
                c1 = [rm_item(c1[0], p1), c1[1]]
                c2 = [c2[0], rm_item(c2[1], p2)]
                count = count+1

    # c2 pos vs. c1 neg
    for p1 in c1[1]:
        for p2 in c2[0]:
            if p1 == p2:
                c1 = [c1[0], rm_item(c1[1], p1)]
                c2 = [rm_item(c2[0], p2), c2[1]]
                count = count+1

    # check for multiple matches or no match and abort if so.
    if count > 1 or count == 0:
        return False

    # compute resolvent
    c1[0].extend(c2[0])
    c1[1].extend(c2[1])

    # make unique and return

    return mk_unique(c1)

In [ ]:
def null_p(clause):
    '''
    function null_p()

    check if "False" is derived
    - returns True if empty clause
    - returns False if non-empty clause
    '''

    if (type(clause) is list) and len(clause[0]) + len(clause[1]) == 0:
        return True
    else:
        return False

In [ ]:
def print_clause(clause):
    '''
    function print_clause

    See function resolve() for clause representation.

    '''

    s = ""
    for pos in clause[0]:
        s = s + f"{pos} \u2228 "

    for neg in clause[1]:
        s = s + f"\u00AC{neg} \u2228 "

    if len(s) == 0:
        print("NULL")
    else:
        n = len(s)
        print(s[0:(n-2)])

Using the `resolve()` function and the null_p function, implement the two-pointer method for propositional logic theorem proving
- use set of support

Theorem representation:  
`[ [<index> <clause1>], [<index> <clause2>], ... ]`

index: integer

clause representation:  
`[<positive-proposition-list>, <negative-proposition-list>]`

proposition-list representation:  
`[ 'p', 'q', 'r' ]`

In [ ]:
# define your theorem, as a set of indexed clauses
theorem = [    # pos-list      # neg-list
           [0, [['p','q'],    ['r']]],       # clause 0     P \/ Q \/ ~R
           [1, [['s'],        ['q']]],       # clause 1     S \/ ~Q
           [2, [['t'],        ['s']]],       # clause 2     T \/ ~S
           [3, [[],           ['p']]],       # clause 3     ~P   - negated conclusion starts here (goal clause index = 3)
           [4, [['r'],        []]],          # clause 4     R
           [5, [[],           ['t']]]        # clause 5     ~T
          ]

# Homework Theorem
theorem2 = [
            [0, [['l'], ['k']]],        # L(Garfield, x) ∨ ¬K(x)
            [1, [['l'], ['L', 'b']]],   # L(x, y) ∨ ¬L(x, Snoopy) ∨ ¬B(y)
            [2, [['B'], []]],           # B(Underdog)
            [3, [[],    ['l']]],        # ¬L(Garfield, α)
            [4, ['k']]                  # ¬K(Garfield)  - negated conclusion starts here (goal index = 4)
           ]

# Theorem from the Exam
theorem3 = [
            [0, [['s', 'q', 'w'],   []]],           # S ∨ Q ∨ W
            [1, [['w'],             ['r']]],        # W ∨ ¬R
            [2, [[],                ['p', 's']]],   # ¬P ∨ ¬S
            [3, [['r'],             ['q']]],        # R ∨ ¬Q
            [4, [['p'],             []]],           # P         - negated conclusion starts here (goal clause index = 4)
            [5, [[],                ['w']]],        # ¬W
           ]

In [ ]:
def prover(thm, goal):
    '''
    function prover: implement this

    two-pointer method, with set of support
    - arguments:
        thm : theorem
        goal : integer index (clause number where the negated conclusion starts)
    - show resolution steps
    - if null_p checks, return True (theorem proven)
    - otherwise return False
    '''

    print('\nprover():\n\nTheorem:')

    for clause in thm:
            print("clause "+str(clause[0])+": ", end="")
            print_clause(clause[1])

    print('Goal clause index=', goal)

    resolved_clauses = thm.copy()  # Copy of the theorem clauses

    # Continue resolving clauses until no further resolutions are possible
    while True:
        new_resolved_clauses = []  # To store newly resolved clauses in this iteration

        for i, clause_i in enumerate(resolved_clauses):
            for j, clause_j in enumerate(resolved_clauses[i + 1:], start=i + 1):
                resolvent = resolve(clause_i[1], clause_j[1])

                if resolvent:  # If resolution is possible
                    print(f'Resolving clause {clause_i[0]} and clause {clause_j[0]}')
                    print_clause(resolvent)

                    # Check if the resolvent is a null clause
                    if null_p(resolvent):
                        print("Null clause found. The theorem is proven.")
                        return True

                    # Add the resolvent to the list of newly resolved clauses
                    new_resolved_clauses.append([len(resolved_clauses) + len(new_resolved_clauses), resolvent])


        # Check if no further resolutions are possible
        if not new_resolved_clauses:
            print("No further resolutions possible. The theorem cannot be proven.")
            return False

        resolved_clauses.extend(new_resolved_clauses)

    return True


In [ ]:
prover(theorem, 3)

In [ ]:
prover(theorem3, 4)